In [ ]:
import os
from google.cloud import vision
import glob
import json
import csv

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
# file require to provide credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='crack-triode-300107-a21d9df5d532.json'
client=vision.ImageAnnotatorClient()

In [ ]:
user_no=11
# path for image files
path="/content/drive/MyDrive/Sample_test/"
# specify features to extract
csv_columns=['FileName', 'bounding_poly', 'fd_bounding_poly', 'LEFT_EYE', 'RIGHT_EYE', 
             'LEFT_OF_LEFT_EYEBROW', 'RIGHT_OF_LEFT_EYEBROW', 'LEFT_OF_RIGHT_EYEBROW', 
             'RIGHT_OF_RIGHT_EYEBROW', 'MIDPOINT_BETWEEN_EYES', 'NOSE_TIP', 'UPPER_LIP', 
             'LOWER_LIP', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'MOUTH_CENTER', 
             'NOSE_BOTTOM_RIGHT', 'NOSE_BOTTOM_LEFT', 'NOSE_BOTTOM_CENTER', 
             'LEFT_EYE_TOP_BOUNDARY', 'LEFT_EYE_RIGHT_CORNER', 'LEFT_EYE_BOTTOM_BOUNDARY',
             'LEFT_EYE_LEFT_CORNER', 'RIGHT_EYE_TOP_BOUNDARY', 'RIGHT_EYE_RIGHT_CORNER', 
             'RIGHT_EYE_BOTTOM_BOUNDARY', 'RIGHT_EYE_LEFT_CORNER', 'LEFT_EYEBROW_UPPER_MIDPOINT',
             'RIGHT_EYEBROW_UPPER_MIDPOINT', 'LEFT_EAR_TRAGION', 'RIGHT_EAR_TRAGION', 'FOREHEAD_GLABELLA', 
             'CHIN_GNATHION', 'CHIN_LEFT_GONION', 'CHIN_RIGHT_GONION', 'ThirtyFive', 'ThirtySix', 'roll_angle', 'pan_angle', 'tilt_angle', 'detection_confidence',
             'landmarking_confidence', 'joy_likelihood', 'sorrow_likelihood', 'anger_likelihood', 'surprise_likelihood',
             'emotion','target']
feature_dict=[]
for filename in glob.glob(os.path.join(path,'*.jpg')):
    print(filename)
    with open(filename,'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.face_detection(image=image)
    faceAnnotations=response.face_annotations
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    for face in faceAnnotations:
        row_dict=dict()
        emotion_dict=dict()
        face_vertices = (['(x:{0},y:{1})'.format(vertex.x, vertex.y)
                    for vertex in face.bounding_poly.vertices])
        face_fd_bounding_poly=(['(x:{0},y:{1})'.format(vertex.x, vertex.y)
                    for vertex in face.fd_bounding_poly.vertices])
        face_landmarks=(['(x:{1},y:{2},z:{3})'.format(str(face.landmarks[i].type_),face.landmarks[i].position.x,face.landmarks[i].position.y,face.landmarks[i].position.z)
                    for i in range(len(face.landmarks))])
    
        row_dict["FileName"]=filename
        row_dict["bounding_poly"]=face_vertices
        row_dict["fd_bounding_poly"]=face_fd_bounding_poly
        
        for item in range(len(face_landmarks)):
            if(str(face.landmarks[item].type_)!='35' and str(face.landmarks[item].type_)!='36'):
                row_dict[str(face.landmarks[item].type_).split('.')[1]]=face_landmarks[item]
            elif(str(face.landmarks[item].type_)=='35'):
                row_dict['ThirtyFive']=face_landmarks[item]
            elif(str(face.landmarks[item].type_)=='36'):
                row_dict['ThirtySix']=face_landmarks[item]
                
        row_dict["roll_angle"]=face.roll_angle
        row_dict["pan_angle"]=face.pan_angle
        row_dict["tilt_angle"]=face.tilt_angle
        row_dict["detection_confidence"]=face.detection_confidence
        row_dict["landmarking_confidence"]=face.landmarking_confidence
        row_dict["joy_likelihood"]=likelihood_name[face.joy_likelihood]
        emotion_dict["Happy"]=likelihood_name[face.joy_likelihood] 
        row_dict["sorrow_likelihood"]=likelihood_name[face.sorrow_likelihood]
        emotion_dict["Sad"]=likelihood_name[face.sorrow_likelihood] 
        row_dict["anger_likelihood"]=likelihood_name[face.anger_likelihood]
        emotion_dict["Stressed"]=likelihood_name[face.anger_likelihood]
        row_dict["surprise_likelihood"]=likelihood_name[face.surprise_likelihood]
        for key in emotion_dict:
            if(emotion_dict[key]=="LIKELY" or emotion_dict[key]=="VERY_LIKELY" or emotion_dict[key]=="POSSIBLE"):
                row_dict["emotion"]=key
                break
            else:
                row_dict["emotion"]="Relaxed"
    '''   
    # check photo capture time and label the image with the self-report that is reported in that time     
    keys=filename.split('/')[-1].split('.')[0] 
    print(keys)
    
    typing_emotion_label={"0":"Relaxed", "1":"Stressed", "2":"Happy", "-2":"Sad", "-99":"No response"}
    tap_path="Data/user_01/Tap_Files"
    instance_count = 0
    for tap_file in glob.glob(os.path.join(tap_path, '*.txt')):
        instance_count=0
        f=open(tap_file,'r')
        data = csv.reader(f)
        for row in data:
            typing_emotion=row[4]
            typing_time=row[5]
            x=typing_time.split()
            x[0]=x[0].replace('-','')
            print("x[0]",x[0])  #date on which emotion recorded is taken
            x[1]=x[1].replace(':','')
            x[1]=x[1][:4] 
            # print(x[1]) #time on which emotion recorded is taken
            if (x[0] in keys):
                temp_keys=keys
                print("temp keys",temp_keys)
                keys=keys.split('_')[-1]
                keys=keys[:4]
                if(int(keys) in range(int(x[1])-15,int(x[1])+15)): #time range of +-15 minutes
                    print("keys",keys)
                    row_dict["target"]=typing_emotion_label[typing_emotion]
                    
            break;
        f.close();  
      
    if "target" in row_dict.keys():
        feature_dict.append(row_dict) 
    '''  
    feature_dict.append(row_dict) 
# write features in the file     
csv_file = "dest_folder/Google_Vision/Output1/user_"+str(user_no)+".csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in feature_dict:
            writer.writerow(data)
except IOError:
    print("I/O error")             

S3bucket_data/user_01/IMG_20191102_131135.jpg
IMG_20191102_131135
x[0] 20191015
x[0] 20191029
x[0] 20191024
x[0] 20191029
x[0] 20191016
x[0] 20191031
x[0] 20191014
x[0] 20191014
x[0] 20191030
x[0] 20191030
x[0] 20191014
x[0] 20191102
temp keys IMG_20191102_131135
x[0] 20191030
x[0] 20191102
x[0] 20191025
x[0] 20191024
x[0] 20191103
x[0] 20191031
x[0] 20191029
x[0] 20191029
x[0] 20191029
x[0] 20191031
x[0] 20191015
x[0] 20191030
x[0] 20191016
x[0] 20191103
x[0] 20191023
x[0] 20191014
x[0] 20191029
x[0] 20191030
x[0] 20191020
x[0] 20191014
x[0] 20191017
x[0] 20191014
x[0] 20191031
x[0] 20191020
x[0] 20191019
x[0] 20191023
x[0] 20191015
x[0] 20191102
x[0] 20191020
x[0] 20191016
x[0] 20191029
x[0] 20191030
x[0] 20191020
x[0] 20191103
x[0] 20191021
x[0] 20191021
x[0] 20191031
x[0] 20191102
x[0] 20191103
x[0] 20191016
x[0] 20191016
x[0] 20191031
x[0] 20191031
x[0] 20191016
x[0] 20191020
x[0] 20191103
x[0] 20191019
x[0] 20191015
x[0] 20191014
x[0] 20191015
x[0] 20191023
S3bucket_data/user_01/

KeyboardInterrupt: 